## **LangChain Installed**

In [1]:
!pip install -U -q google-genai

## **OpenAI Client for Gemini Flash**

In [2]:
!pip install google-generativeai
!pip install langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.9 MB/s eta 0:00:00


## **Set Up Environment**

Import the required libraries:

### **Use Secret Key for passing API Keys**

In [3]:
# import langchain.llms
# dir(langchain.llms)

#!python --version


## **Configure the Gemini Flash Model**

Use the `GoogleGeminiFlash` class to interface with the Gemini Flash API. You’ll need to pass your API key.



In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [5]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

#GOOGLE_API_KEY



In [6]:
MODEL: str = "gemini-2.0-flash-exp"

#MODEL

In [7]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [8]:
# Initialize an instance of the ChatGoogleGenerativeAI with specific parameters
llm: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(
    model=MODEL,  # Specify the model to use
    api_key=GOOGLE_API_KEY,     # Provide the Google API key for authentication
    temperature=0.2,            # Set the randomness of the model's responses (0 = deterministic, 1 = very random)
)


## **Create a Prompt Template**

In [9]:
prompt_template = PromptTemplate(
    input_variables=["question"],
    template="""
You are an expert assistant. Your job is to provide concise, accurate answers.
Question: {question}
Answer:
"""
)

## **Build the LangChain Pipeline**

In [10]:
chain = LLMChain(llm=llm, prompt=prompt_template)

<ipython-input-10-4f9708b53e60>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)


## **Run the Basic Example**

In [11]:
# Run the chain with sample question
question = "What is the Agentic AI"
#answer = chain.run({"question":question})
answer = chain.run(question)
print("Answer:",answer)

<ipython-input-11-10159361adad>:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = chain.run(question)


Answer: Agentic AI refers to artificial intelligence systems that can autonomously perceive, reason, plan, and act in an environment to achieve specific goals.



In [12]:
question = "Who is the founder of PIAIC"
answer = chain.run({"question": question})
print("Answer:", answer)

Answer: Zia Khan



In [13]:
question = "What are latest trend in Gen AI"
answer = chain.run({"question": question})
print("Answer:", answer)

Answer: Recent trends in Gen AI include:

*   **Multimodality:** Models handling multiple data types (text, images, audio, video).
*   **Improved Reasoning:** Enhanced ability to perform complex tasks requiring logical inference.
*   **Efficiency:** Smaller, faster models with reduced computational needs.
*   **Personalization:** Tailoring outputs to individual user preferences.
*   **Responsible AI:** Focus on ethics, bias mitigation, and safety.
*   **Edge Deployment:** Running models on devices rather than cloud servers.
*   **Agentic AI:** Autonomous systems that can plan and execute tasks.



## **Add Memory**

Create an instance of ConversationBufferMemory. This stores the conversation history.

In [14]:
from langchain.memory import ConversationBufferMemory

In [15]:
# Set up memory to retain conversation history
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

llm: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(
    model=MODEL,
    api_key=GOOGLE_API_KEY,
    temperature=0.7,
    memory = memory,
    max_tokens=150
)
#Add memory
chain = LLMChain(llm=llm, prompt=prompt_template, memory=memory)

<ipython-input-15-c0f2395cd2f2>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [17]:
# Start interacting
print("Ask your question:")

while True:
    user_input = input("Question: ")
    response = chain.run({"question": user_input})
    print(f"Answer: {response}")

    # Check for a command to stop the loop
    if user_input.lower() in ['exit', 'quit']:
        print("Exiting the program...")
        break  # Stop the loop and exit the program

Ask your question:
Question: exit
Answer: Okay.

Exiting the program...


## Integrate External Tools

### News API

In [18]:
import requests

# Set your News API key here
NEWS_API_KEY = userdata.get('NEWS_API_KEY')

#NEWS_API_KEY

In [19]:
# Example of integrating an external tool (News API)
def get_news(query):
    url = f'https://newsapi.org/v2/everything?q={query}&apiKey={NEWS_API_KEY}'
    response = requests.get(url).json()

    # Check if articles were returned
    if 'articles' in response and response['articles']:
        # Extract top 3 headlines
        top_articles = response['articles'][:3]
        news_response = "\n".join([f"{i+1}. {article['title']} - {article['source']['name']}" for i, article in enumerate(top_articles)])
        return news_response
    else:
        return "No news articles found at the moment."

# Initialize the LLMChain with memory and the prompt
chain = LLMChain(llm=llm, prompt=prompt_template, memory=memory)

# Example conversation loop
while True:
    user_input = input("Question: ")  # Ask the user for a question

    # Check for a command to stop the loop
    if user_input.lower() in ['exit', 'quit']:
        print("Exiting the program...")
        break  # Stop the loop and exit the program

    # Check if the user is asking for news
    if "news" in user_input.lower():
        # Extract query term (topic) from the user's input
        query = user_input.split("about")[-1].strip()  # Simple extraction of topic after 'about'
        news_response = get_news(query)  # Get news articles
        print(f"News: {news_response}")  # Print news articles
    else:
        # Otherwise, run the chain with the user input
        response = chain.run({"question": user_input})
        print(f"Answer: {response}")


Question: Latest technology news
News: 1. Tim Cook is the latest tech CEO to meet with Trump at Mar-a-Lago - The Verge
2. OpenAI sued by Canada’s biggest media outlets - The Verge
3. The biggest news from Amazon re:Invent 2024 - VentureBeat
Question: exit
Exiting the program...


### Weather API

In [20]:
# Set your Weather API key here
WEATHER_API_KEY = userdata.get('WEATHER_API_KEY')  # Replace with your actual Weather API key

# Function to get weather for a specific location
def get_weather(location):
    # Define the API endpoint and query parameters
    url = f"http://api.weatherapi.com/v1/current.json?key={WEATHER_API_KEY}&q={location}"

    # Send a GET request to the Weather API
    response = requests.get(url).json()

    # Check if the response contains valid data
    if 'current' in response:
        temperature = response['current']['temp_c']
        condition = response['current']['condition']['text']
        humidity = response['current']['humidity']
        wind_speed = response['current']['wind_kph']

        # Return formatted weather information
        return (f"The current temperature in {location} is {temperature}°C with {condition}. "
                f"Humidity is {humidity}% and the wind speed is {wind_speed} km/h.")
    else:
        # If the location was not found or another error occurred
        return "Could not fetch the weather data. Please try again later."

# Main loop to ask for user input and fetch weather data
while True:
    user_input = input("Enter a location to get the weather (or type 'exit' to quit): ")

    # Exit condition
    if user_input.lower() == 'exit':
        print("Exiting the program...")
        break

    # Get weather for the location entered by the user
    weather_info = get_weather(user_input)

    # Print the weather information
    print(weather_info)

Enter a location to get the weather (or type 'exit' to quit): Islamabad
The current temperature in Islamabad is 8.7°C with Clear. Humidity is 58% and the wind speed is 8.3 km/h.
Enter a location to get the weather (or type 'exit' to quit): Lahore
The current temperature in Lahore is 12.3°C with Mist. Humidity is 82% and the wind speed is 6.1 km/h.
Enter a location to get the weather (or type 'exit' to quit): exit
Exiting the program...
